In [3]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     api_key=api_key
# )
model_client = OllamaChatCompletionClient(
    model="llama3.2"
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [4]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=109), metadata={}, content="As summer's warmth begins to fade,\nA new season dawns, with colors displayed.\nGolden hues and crimson bright,\nThe leaves rustle, in autumn's delight.\n\nThe air is crisp, the wind is cold,\nNature's canvas, young and old.\nOrchards ripe, with fruit so sweet,\nThe scent of woodsmoke wafts at our feet.\n\nThe trees stand tall, in golden pride,\nTheir branches swaying, side by side.\nA fleeting glimpse, of beauty rare,\nFall's seasonal charm, beyond compare.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=296), metadata={}, content='Your poem effectively captures the essence of the fall season, conveying a sense of transition from warm summer days to 

### Observing a Team

In [6]:
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=108) metadata={} content="As summer's warmth begins to fade,\nThe autumn leaves their vibrant shade,\nGolden hues and crimson bright,\nA colorful display, a wondrous sight.\n\nThe air is crisp, the winds do blow,\nAnd nature's final dance begins to show,\nThe trees stand tall, their branches high,\nA canopy of color in the autumn sky.\n\nThe scent of woodsmoke wafts through air,\nAs earthy smells and forest secrets share,\nA season of change, a time to roam,\nIn autumn's beauty, we find our home." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=160, completion_tokens=276) metadata={} content='Your poem is well-written and effectively captures the essence of the fall season. Here are some suggestions for improvement:\n\n* Consider adding more sensory details to help bri

In [7]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
As autumn leaves begin to fall,
Golden hues and crimson all,
The air is crisp, the winds do blow,
A season's farewell, as summer goes.

The trees stand tall, their branches gray,
Their limbs outstretched, in a final sway,
The forest floor, a carpet deep,
With leaves that rustle, and softly creep.

The scent of woodsmoke wafts through air,
As nature's final dance, is beyond compare,
A time for change, for growth to cease,
And winter's slumber, to bring release.
---------- critic ----------
Your poem effectively captures the essence of the fall season, conveying a sense of melancholy and transition. Here are some specific points that stand out:

1. The imagery is vivid and evocative, particularly in lines such as "Golden hues and crimson all," "The forest floor, a carpet deep," and "The scent of woodsmoke wafts through air." These descriptions create a strong sense of atmosphere and immersi

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=112), metadata={}, content="As autumn leaves begin to fall,\nGolden hues and crimson all,\nThe air is crisp, the winds do blow,\nA season's farewell, as summer goes.\n\nThe trees stand tall, their branches gray,\nTheir limbs outstretched, in a final sway,\nThe forest floor, a carpet deep,\nWith leaves that rustle, and softly creep.\n\nThe scent of woodsmoke wafts through air,\nAs nature's final dance, is beyond compare,\nA time for change, for growth to cease,\nAnd winter's slumber, to bring release.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=299), metadata={}, content='Your poem effectively captures the essence of the fall season, conveying a sense of melancholy and transi